# Creation of DataFile

In [1]:
#Creating ML.csv File to be used below
import pandas as pd
import numpy as np
import random

pd.options.mode.chained_assignment = None
final_file=pd.read_csv('ML.csv')
i=1
for name in range(1,6):
    print(name)
    data=pd.read_csv('democ'+str(name)+'.csv')
    rows=len(data)
    index_set=set()
    while len(index_set)<3000:
        index=random.randint(0, rows-1)
        if index not in index_set:
            index_set.add(index)
            final_file['Tweet'][i]=data['tweetText'][index]
            final_file['Party'][i]=0
            i+=1
        else:
            continue
for name in range(1,6):
    print(name)
    data=pd.read_csv('repub'+str(name)+'.csv')
    rows=len(data)
    index_set=set()
    while len(index_set)<3000:
        index=random.randint(0, rows-1)
        if index not in index_set:
            index_set.add(index)
            final_file['Tweet'][i]=data['tweetText'][index]
            final_file['Party'][i]=1
            i+=1
        else:
            continue
final_file.to_csv('ML.csv',index='False')


1
2
3
4
5
1
2
3
4
5


# PreProcessing

In [2]:
#Performing Preprocessing
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

pd.options.mode.chained_assignment = None
nltk.download('stopwords')
csvname='ML.csv'
data=pd.read_csv(csvname)
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
rows=len(data)

# loop to clean the tweets and to remove stopwords and perform stemming.
for i in range (0,rows):
    oldtext=data['Tweet'][i] 
    newtext=' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(oldtext)).split()) # removes hashtags, https links, usernames and any other alpha-numeric characters.
    newtext= re.sub('[^a-zA-Z]'," ",str(newtext)) #removes all other characters other than alphabets
    newtext=newtext.lower()
    newtext=newtext.split()
    ps=PorterStemmer()
    newtext= [ps.stem(word) for word in newtext if word not in set(stopwords.words('english'))]
    newtext=' '.join(newtext)
    #print(newtext)
    data['Tweet'][i]=newtext
        
# Saving changes to the csv file
data.to_csv(csvname,index='False')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Machine Learning

In [3]:
import re
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking

pd.options.mode.chained_assignment = None
csvname="ML.csv" # Insert your csv name here.
data=pd.read_csv(csvname)
rows=len(data)

# 'corpus' list consists of the tweet text and 'y' consists of the political affliation of the tweet.

corpus=[]
y=[]
for i in range(rows):
    newtext=data['Tweet'][i]
    if(len(str((newtext)))!=0 and str(newtext)!='nan'):
        corpus.append(newtext)
        y.append(data['Party'][i])

In [4]:
# Using Term-Frequency-Iverse-Document-Frequency and fitting corpus into it.
# Data-set 1 (X1) created using TF-IDF
vectorizer = TfidfVectorizer(min_df=50,max_features=1500)
vectorizer.fit(corpus)
X1 = vectorizer.transform(corpus)
X1=X1.toarray()

# Using Count-Vectorizer and fitting corpus into it.
# Data-set 2 (X2) created using Count-Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X2 = cv.fit_transform(corpus).toarray()
y = np.array(y)

In [5]:
# Splitting the dataset into Training and Test set.
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)
from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size = 0.2, random_state = 0)


## Linear Discriminant Analysis

In [6]:
# Performing LinearDiscriminantAnalysis.
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components =1)
X1_train = lda.fit_transform(X1_train, y1_train)
X1_test = lda.transform(X1_test)
lda = LDA(n_components =1)
X2_train = lda.fit_transform(X2_train, y2_train)
X2_test = lda.transform(X2_test)

## Support Vector Classification

In [7]:
# Fitting the data into Machine Learning Models.

from sklearn.svm import SVC
from sklearn import svm
SVC1 = SVC(kernel = 'linear', random_state = 0)
SVC1.fit(X1_train, y1_train)

SVC2 = SVC(kernel = 'linear', random_state = 0)
SVC2.fit(X2_train, y2_train)

SVC(kernel='linear', random_state=0)

## AdaBoost Classifier

In [8]:
## Fitting the data into AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
adaboostSVC1 = AdaBoostClassifier(n_estimators=50, base_estimator= SVC1,learning_rate=1, random_state = 1,algorithm='SAMME')
adaboostSVC1.fit(X1_train,y1_train)

adaboostSVC2 = AdaBoostClassifier(n_estimators=50, base_estimator= SVC2,learning_rate=1, random_state = 1,algorithm='SAMME')
adaboostSVC2.fit(X2_train,y2_train)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=SVC(kernel='linear', random_state=0),
                   learning_rate=1, random_state=1)

## XGBoost Classifier

In [9]:
# Fitting the data into XGBoost Classifier
from xgboost import XGBClassifier
XGB1 = XGBClassifier()
XGB1.fit(X1_train, y1_train)

XGB2 = XGBClassifier()
XGB2.fit(X2_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Random Forest Classifier

In [10]:
# Fitting the data into Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
RF1 = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
RF1.fit(X1_train, y1_train)
RF2 = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
RF2.fit(X2_train, y2_train)

RandomForestClassifier(criterion='entropy', random_state=0)

## Decision Tree Classifier

In [11]:
#Fitting the data into Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
DT1 = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DT1.fit(X1_train, y1_train)

DT2= DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DT2.fit(X2_train, y2_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

# Results of Predictions

In [15]:
#Predicting Outputs of the different classifiers

y_pred=DT1.predict(X1_test)
print("The Accuracy using Decision Tree on Data-set 1: ",accuracy_score(y1_test,y_pred))

y_pred=DT2.predict(X2_test)
print("The Accuracy using Decision Tree on Data-set 2: ",accuracy_score(y2_test,y_pred)) 

y_pred=XGB1.predict(X1_test)
print("The Accuracy using Extreme Gradient Boost on Data-set 1: ",accuracy_score(y1_test,y_pred))

y_pred=XGB2.predict(X2_test)
print("The Accuracy using Extreme Gradient Boost on Data-set 2: ",accuracy_score(y2_test,y_pred)) 

y_pred=RF1.predict(X1_test)
print("The Accuracy using Random Forest on Data-set 1: ",accuracy_score(y1_test,y_pred)) 

y_pred=RF2.predict(X2_test)
print("The Accuracy using Random Forest on Data-set 2: ",accuracy_score(y2_test,y_pred))

y_pred=SVC1.predict(X1_test)
print("The Accuracy using Support Vector Clustering on Data-set 1: ",accuracy_score(y1_test,y_pred))

y_pred=SVC2.predict(X2_test)
print("The Accuracy using Support Vector Clustering on Data-set 2: ",accuracy_score(y2_test,y_pred))

y_pred=adaboostSVC1.predict(X1_test)
print("The Accuracy using AdaBoosted Support Vector Clustering on Data-set 1: ",accuracy_score(y1_test,y_pred))

y_pred=adaboostSVC2.predict(X2_test)
print("The Accuracy using AdaBoosted Support Vector Clustering on Data-set 2: ",accuracy_score(y2_test,y_pred))

The Accuracy using Decision Tree on Data-set 1:  0.6096498575975875
The Accuracy using Decision Tree on Data-set 2:  0.6191991958452002
The Accuracy using Extreme Gradient Boost on Data-set 1:  0.6463394203384152
The Accuracy using Extreme Gradient Boost on Data-set 2:  0.6557212263360697
The Accuracy using Random Forest on Data-set 1:  0.6119953090970012
The Accuracy using Random Forest on Data-set 2:  0.6193667280951584
The Accuracy using Support Vector Clustering on Data-set 1:  0.6513653878371587
The Accuracy using Support Vector Clustering on Data-set 2:  0.6580666778354833
The Accuracy using AdaBoosted Support Vector Clustering on Data-set 1:  0.6481822750879545
The Accuracy using AdaBoosted Support Vector Clustering on Data-set 2:  0.6560562908359859


## Voting Classifier

In [16]:
# Performing Voting to improve accuracy. 
from sklearn.ensemble import VotingClassifier

eclf1 = VotingClassifier(estimators=[
     ('RF1',RF1),('adaboostSVC1',adaboostSVC1),('XGB1',XGB1)], voting='soft')
eclf1.fit(X1_train, y1_train)
y1_pred=eclf1.predict(X1_test)
print("The Accuracy using Voting Classifier on Data-set 1: ",accuracy_score(y1_test,y1_pred)) 

eclf2 = VotingClassifier(estimators=[
     ('RF2',RF2),('adaboostSVC2',adaboostSVC2),('XGB2',XGB2)], voting='soft')
eclf2.fit(X2_train, y2_train)
y2_pred=eclf2.predict(X2_test)
print("The Accuracy using Voting Classifier on Data-set 2: ",accuracy_score(y2_test,y2_pred))

The Accuracy using Voting Classifier on Data-set 1:  0.6414809850896298
The Accuracy using Voting Classifier on Data-set 2:  0.6525381135868654
